In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
import random
import statsmodels.tsa.api as smt
import warnings
warnings.filterwarnings("ignore")

from matplotlib.pyplot import MultipleLocator

In [2]:
def API(list_ = []):

    n = len(list_)
    if n < 2:
        return np.nan

    mean_list = np.mean(list_)

    numerator = np.sum((list_[:-1] - mean_list) * (list_[1:] - mean_list))/(n-1)
    denominator = np.sum((list_ - mean_list) ** 2)/(n)

    if denominator == 0:
        return np.nan

    result = numerator / denominator

    return result

In [3]:
def HUPSPI(sequence):
    N_CAV = 0  # Total number of CAVs
    platoon_count = 0  # Count of platoons

    i = 0
    while i < len(sequence):
        if sequence[i] == 1:  # Start of a CAV platoon
            platoon_size = 0
            while i < len(sequence) and sequence[i] == 1:
                platoon_size += 1
                i += 1
            N_CAV += platoon_size  # Add the size of this platoon to N_CAV
            platoon_count += 1  # Increase the count of platoons
        else:
            i += 1

    if platoon_count == 0:  # If there are no platoons, return 0 to avoid division by zero
        return 0

    HUPSPI = N_CAV / platoon_count  # Calculate HUPSPI

    return HUPSPI

In [4]:
def MCPI(sequence):
    N = len(sequence)
    P_1 = np.mean(sequence)  # Penetration rate of CAVs

    # Count the number of each vehicle pair
    N00 = N01 = N10 = N11 = 0
    for i in range(1, N):
        if sequence[i-1] == 0 and sequence[i] == 0:
            N00 += 1
        elif sequence[i-1] == 0 and sequence[i] == 1:
            N01 += 1
        elif sequence[i-1] == 1 and sequence[i] == 0:
            N10 += 1
        elif sequence[i-1] == 1 and sequence[i] == 1:
            N11 += 1

    if N == 1:  # Edge case where N=1, MCPI is undefined
        return None

    # Denominators for the first set of formulas
    denom_1 = P_1 * (1 - P_1) * (N - 1)

    # Calculate O11, O10, O01, O00 using the first set of formulas
    O11 = (N11 / denom_1) - (P_1 / (1 - P_1))
    O10 = 1 - (N10 / denom_1)
    O01 = 1 - (N01 / denom_1)
    O00 = (N00 / denom_1) - ((1 - P_1) / P_1)

    # If any O_sr is negative, recalculate it using the second set of formulas
    min_val = min(P_1, 1 - P_1) - P_1 * (1 - P_1)

    if O11 < 0:
        O11 = (N11 / (N - 1) - P_1**2) / min_val
    if O10 < 0:
        O10 = (P_1 * (1 - P_1) - N10 / (N - 1)) / min_val
    if O01 < 0:
        O01 = (P_1 * (1 - P_1) - N01 / (N - 1)) / min_val
    if O00 < 0:
        O00 = (N00 / (N - 1) - (1 - P_1)**2) / min_val

    # Calculate MCPI as the average of O11, O10, O01, and O00
    MCPI = np.mean([O11, O10, O01, O00])

    return MCPI

In [5]:
def HEPSPI(sequence):
    N11 = N01 = 0

    for i in range(1, len(sequence)):
        if sequence[i-1] == 1 and sequence[i] == 1:
            N11 += 1
        elif sequence[i-1] == 0 and sequence[i] == 1:
            N01 += 1

    # To avoid division by zero, return 0 if there are no CAVs (i.e., N11 + N01 == 0)
    if N11 + N01 == 0:
        return 0

    HEPSPI_value = N11 / (N11 + N01)

    return HEPSPI_value

In [6]:
def JPSPI(sequence):
    N_CAV = sequence.count(1)  # Total number of CAVs in the sequence
    if N_CAV == 0:  # If there are no CAVs, JPSPI is undefined
        return 0

    platoon_sizes = []  # List to store sizes of each CAV platoon

    i = 0
    while i < len(sequence):
        if sequence[i] == 1:  # Start of a CAV platoon
            platoon_size = 0
            while i < len(sequence) and sequence[i] == 1:
                platoon_size += 1
                i += 1
            platoon_sizes.append(platoon_size)
        else:
            i += 1

    # Count the number of platoons of each size
    m_j = [size for size in platoon_sizes if size >= 2]  # Filter platoons of size >= 2
    total_cavs_in_platoons = sum(m_j)  # Total number of CAVs in platoons of size >= 2

    # Calculate JPSPI
    JPSPI_value = total_cavs_in_platoons / N_CAV if N_CAV > 0 else 0

    return JPSPI_value

In [7]:
def WPSPI(sequence):
    platoon_sizes = []  # List to store sizes of each CAV platoon

    i = 0
    while i < len(sequence):
        if sequence[i] == 1:  # Start of a CAV platoon
            platoon_size = 0
            while i < len(sequence) and sequence[i] == 1:
                platoon_size += 1
                i += 1
            platoon_sizes.append(platoon_size)
        else:
            i += 1

    M = len(platoon_sizes)  # Total number of CAV platoons
    if M == 0:  # If there are no CAV platoons, return 0 to avoid division by zero
        return 0

    # Calculate WPSPI as the average platoon size
    WPSPI_value = sum(platoon_sizes) / M

    return WPSPI_value

In [8]:
import itertools

def generate_all_sequences(length=10):
    # Generate all binary sequences of the given length
    all_sequences = list(itertools.product([0, 1], repeat=length))

    # Filter out the all 0s and all 1s sequences
    valid_sequences = [
        seq for seq in all_sequences if not all(x == 0 for x in seq) and not all(x == 1 for x in seq)
    ]

    # Calculate the penetration rate for each sequence
    sequences_with_penetration = [
        (list(seq), length, sum(seq) / length) for seq in valid_sequences
    ]

    return sequences_with_penetration

sequences = generate_all_sequences()

In [9]:
sequences = generate_all_sequences()

results = []
for seq, length, penetration_rate in sequences:
    result = {
        'Sequence': seq,
        'Length': length,
        'Penetration Rate': penetration_rate,
        'API': API(seq),
        'HUPSPI': HUPSPI(seq),
        'MCPI': MCPI(seq),
        'HEPSPI': HEPSPI(seq),
        'JPSPI': JPSPI(seq),
        'WPSPI': WPSPI(seq)
    }
    results.append(result)
df_results = pd.DataFrame(results)

In [10]:
df_results[['HUPSPI','WPSPI','MCPI','API']].describe()

,HUPSPI,WPSPI,MCPI,API
count,1022.000000,1022.000000,1022.000000,1022.000000
mean,1.982387,1.982387,-0.313535,-0.111111
std,1.008962,1.008962,0.569788,0.317829
min,1.000000,1.000000,-2.111111,-1.000000
25%,1.333333,1.333333,-0.583333,-0.351852
50%,1.666667,1.666667,-0.245370,-0.166667
75%,2.333333,2.333333,0.111111,0.111111
max,9.000000,9.000000,0.777778,0.777778
